In [1]:
!pip install mlflow
!pip install tensorflow tensorflow-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.5/677.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os
import torch
from bs4 import BeautifulSoup

import mlflow
import mlflow.sklearn
import shutil

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_score

import tensorflow_hub as hub
import tensorflow as tf

import pickle

In [3]:
os.makedirs("/kaggle/working/models", exist_ok=True)

In [4]:
mlflow.set_tracking_uri("/kaggle/working/mlruns")

mlflow.set_experiment("USE_Model")

print("MLflow configuré sur Kaggle ! Logs enregistrés dans /kaggle/working/mlruns")

2025/03/14 00:49:42 INFO mlflow.tracking.fluent: Experiment with name 'USE_Model' does not exist. Creating a new experiment.


MLflow configuré sur Kaggle ! Logs enregistrés dans /kaggle/working/mlruns


In [5]:
data = pd.read_csv("/kaggle/input/bigot-benjamin-1-notebook-exploration-022025/preprocessed_data.csv")
data.head()

,Id,Title,Body,Tags,Score,ViewCount,AnswerCount,CreationDate,Processed
0,75956209,"Error ""'DataFrame' object has no attribute 'ap...",<p>I am trying to append a dictionary to a Dat...,<python><pandas><dataframe><concatenation><att...,312,692073,3,2023-04-07 07:05:59,"['error', 'object', 'attribute', 'append', 'tr..."
1,75898276,"OpenAI API error 429: ""You exceeded your curre...",<p>I'm making a Python script to use OpenAI vi...,<python><prompt><openai-api><completion><chatg...,169,416362,5,2023-03-31 11:58:04,"['api', 'error', 'exceed', 'quota', 'check', '..."
2,77171270,Compilation error after upgrading to JDK 21 - ...,"<p>After upgrading to JDK 21, I have the follo...",<spring-boot><compiler-errors><upgrade><lombok...,358,396585,13,2023-09-25 09:05:11,"['compilation', 'error', 'upgrade', 'upgrade',..."
3,76435306,"babel-preset-react-app, is importing the ""@bab...",<p>My issue is that I've tried to create a new...,<javascript><node.js><reactjs><babeljs><ejs>,155,273911,24,2023-06-08 19:56:44,"['react', 'app', 'import', 'babel', 'plugin', ..."
4,76303988,"""ERROR: File xxx can't be indexed twice."" when...",<p>Below is my Azure pipeline to analyze a sol...,<c#><c++><sonarqube><sonarqube-scan><sonar-run...,0,208657,1,2023-05-22 07:45:00,"['error', 'file', 'index', 'analyze', 'solutio..."


In [6]:
df_train = pd.read_csv("/kaggle/input/embeddings/X_train.csv")
df_test = pd.read_csv("/kaggle/input/embeddings/X_test.csv")

X_train = df_train.to_numpy()
X_test = df_test.to_numpy()

In [7]:
df_train_use, df_test_use, df_y_train, df_y_test = pd.read_csv("/kaggle/input/embeddings/X_train_use.csv"), \
pd.read_csv("/kaggle/input/embeddings/X_test_use.csv"), \
pd.read_csv("/kaggle/input/embeddings/y_train.csv"), \
pd.read_csv("/kaggle/input/embeddings/y_test.csv")

X_train_use, X_test_use, y_train, y_test = df_train_use.to_numpy(), df_test_use.to_numpy(), df_y_train.to_numpy(), df_y_test.to_numpy()

# Vérifier la taille des jeux de données
print(f"Taille du train : {len(X_train_use)}")
print(f"Taille du test : {len(X_test_use)}")

Taille du train : 8000
Taille du test : 2000


In [8]:
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train_use, y_train)

y_pred = clf.predict(X_test_use)
score = jaccard_score(y_test, y_pred, average="samples")
print("Jaccard Score avec Word2Vec :", score)

with mlflow.start_run():
    mlflow.log_param("model", "USE : OneVsRest + LogisticRegression")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_metric("jaccard_score", score)
    
    # Sauvegarder le modèle
    mlflow.sklearn.log_model(clf, "model")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Jaccard Score avec Word2Vec : 0.12149285714285714


2025/03/14 00:53:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [9]:
clf_sgd = OneVsRestClassifier(SGDClassifier(loss="log_loss", max_iter=1000))
clf_sgd.fit(X_train_use, y_train)

y_pred = clf_sgd.predict(X_test_use)
score = jaccard_score(y_test, y_pred, average="samples")
print("Jaccard Score avec Word2Vec :", score)

with mlflow.start_run():
    mlflow.log_param("model", "USE : OneVsRest + SGDClassifier")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_metric("jaccard_score", score)
    
    # Sauvegarder le modèle
    mlflow.sklearn.log_model(clf_sgd, "model")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Jaccard Score avec Word2Vec : 0.13223095238095237


2025/03/14 00:57:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [10]:
y_proba = clf_sgd.predict_proba(X_test_use)

with open("/kaggle/input/embeddings/mlb.pkl", "rb") as f:
    mlb_loaded = pickle.load(f)

print("Classes rechargées :", mlb_loaded.classes_)
# Définir un seuil de probabilité
threshold = 0.3

# Fonction pour récupérer les tags qui dépassent le seuil
def get_tags_with_threshold(probas, threshold=0.3):
    indices = np.where(probas >= threshold)[0]
    if len(indices) == 0:
        indices = [np.argmax(probas)]
    return [mlb_loaded.classes_[i] for i in indices]

# Appliquer sur toutes les prédictions
predicted_tags = [get_tags_with_threshold(proba, threshold) for proba in y_proba]


Classes rechargées : ['python' 'javascript' 'reactjs' ... 'quic' 'apple-developer' 'appkit']


In [11]:
X_test_with_predictions = pd.DataFrame(X_test)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train) 
y_test = pd.DataFrame(y_test)

X_test_with_predictions["Predicted_Tags"] = predicted_tags

X_total = pd.concat([X_train, X_test], axis=0).reset_index(drop=True)
Y_total = pd.concat([y_train, y_test], axis=0).reset_index(drop=True)
X_total_reset = X_total.reset_index(drop=True)
Y_total_reset = Y_total.reset_index(drop=True)

# Concaténer horizontalement (axis=1)
df_total = pd.concat([X_total_reset, Y_total_reset], axis=1)

X_test_with_predictions.to_csv("X_test_with_predictions_use.csv", index=False)
df_total.to_csv("df_final_use.csv", index=False)

In [12]:
# Compresser tout le répertoire mlruns
shutil.make_archive("/kaggle/working/mlruns_USE", "zip", "/kaggle/working/mlruns")

'/kaggle/working/mlruns_USE.zip'